The following notebook demonstrates the main steps of the pipeline:
1. Setup (Defining the model and parameters)
2. Training the model
3. Testing the model

At its core, the `main.py` module does the exact same thing here with extra steps (for file and user handling).

## Setup
First, we need to setup the training configuration. This includes defining the training/validation/testing datasets, defining the model, number of epochs, the optimiser/loss function, etc. All these can be neatly packed into a single object called [TrainingConfig](../src/types.py). 

In [ ]:
import os
import sys
import logging
import pickle
import torch
import torch.nn as nn
import numpy as np

import scripts.data_loader as data_loader
import src.training as training
import src.evaluation as evaluation

from dataclasses import dataclass, asdict
from src.types import *
from src.models.main_model import OB_05Model
from src.models.main_model_v1 import OB_05Model_Variant1
from src.models.main_model_v2 import OB_05Model_Variant2
from scripts.visualization.model_evaluation import TrainingVisualizations, TestingVisualizations


output_dir = r"../output/pipeline_demo/"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)


# Initialize datasets
training_dataset, validation_dataset, testing_dataset = data_loader.split_images_dataset()

training_set_loader = data_loader.create_data_loader(training_dataset)
validation_set_loader = data_loader.create_data_loader(validation_dataset)
testing_set_loader = data_loader.create_data_loader(testing_dataset)

In [ ]:
# logger for output (we can output training data to stdout or a file for example)
logger = logging.getLogger()
logger.setLevel(logging.INFO)
console_handler = logging.StreamHandler(sys.stdout)
logger.addHandler(console_handler)

# can pick any model
model = OB_05Model()
# model = OB_05Model_Variant1()
# model = OB_05Model_Variant2()

initial_learning_rate = 0.0001
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=initial_learning_rate, weight_decay=5e-2)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.1, patience=5)

training_config = training.TrainingConfig(
    model_name="pipeline_demo",
    output_dir=output_dir,
    output_logger=logger,

    training_set_loader=training_set_loader,
    validation_set_loader=validation_set_loader,
    testing_set_loader=testing_set_loader,

    epochs=10,

    classes=data_loader.get_trainset().classes,
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler
)

## Training &amp; Visualizations

In [ ]:
training_logger = training.train_model(training_config)

# save the training logger -> we can visualize data without having to re-train the model
training_logger_path = os.path.join(output_dir, "training_logger.pkl")
with open(training_logger_path, "wb") as file:
    pickle.dump(training_logger, file)

# save the model so we can test it without having to re-train the model
torch.save(model.state_dict(), os.path.join(output_dir, "model.pth"))

Visualization functions are located in the [TrainingVisualizations](../scripts/visualization/model_evaluation.py) script/module.

In [ ]:
fig = TrainingVisualizations.plot_training_metrics(training_logger)

# Testing & Visualizations

In [ ]:
evaluation_results = evaluation.evaluate_model(logger, model, testing_set_loader)

Visualization functions are located in the [TestingVisualizations](../scripts/visualization/model_evaluation.py) script/module.


In [ ]:
# Generated confusion matrix from the test set
_ = TestingVisualizations.generate_confusion_matrix_table(evaluation_results)

In [ ]:
# Overall metrics table from the test set
# Displays macro and micro metrics
_ = TestingVisualizations.generate_overall_metrics_table(evaluation_results)

In [ ]:
# Displays the metrics per emotion class AS A TABLE
_ = TestingVisualizations.generate_metrics_per_class_table(evaluation_results)

In [ ]:
# Displays the metrics per emotion class AS A BAR PLOT
_ = TestingVisualizations.plot_metrics_per_class(evaluation_results)

## Additional functionalities
Metrics are calculated using a confusion table. Since we have 4 classes (angry, engaged, happy, neutral), our confusion matrix is a 4x4 matrix. For information on how metrics are calculated, see [this](https://akash-borgalli.medium.com/confusion-matrix-for-n-x-n-matrix-488e8ff18321) article.

The [ConfusionMatrix](../src/utils/confusion_matrix.py) class contains the methods for calculating not only the overall metrics for the model, but the metrics for each individual class. If you want to perform extra processing or visualizations on the `EvaluationResults` objects returned from testing, there are some pre-defined methods that you can use as seen below:

In [ ]:
evaluation_results.get_metrics_per_class_as_df()

In [ ]:
evaluation_results.get_confusion_matrix_as_df()

In [ ]:
evaluation_results.get_metrics_table_as_df()

You can format the above dataframe into a better visualization as seen below. Note that you can only do this in Jupyter notebooks (can't do it in the console)

In [ ]:
confusion_matrix = evaluation_results.confusion_matrix

macro_precision, macro_recall, macro_f1_score, macro_accuracy = cm_macro.calculate_overall_metrics(confusion_matrix)
micro_precision, micro_recall, micro_f1_score, micro_accuracy = cm_micro.calculate_overall_metrics(confusion_matrix)
data = [[macro_precision, macro_recall, macro_f1_score, micro_precision, micro_recall, micro_f1_score, (macro_accuracy + micro_accuracy)]]
tuples = [("macro", "precision"), ("macro", "recall"), ("macro", "f1_score"), ("micro", "precision"), ("micro", "recall"), ("micro", "f1_score"), ("", "accuracy")]

df = pd.DataFrame(data, index=pd.Index(["model"]), columns=pd.MultiIndex.from_tuples(tuples, names=["", "metrics"]))
df.style

### Changing the training/testing
All models are trained and tested the same way for more accurate comparisons between them. The training portion of the application happens in the [training.py](../src/training.py) module, while the testing/evaluation part happens in the [evaluation.py](../src/evaluation.py) module.

### Adding models
You can define your own models in the `src/models` directory. You can then train and test these models by placing them in the training config.

End